In [1]:
import numpy as np
from math import cos, sin, radians, sqrt, pi, atan, tan


In [9]:
class Elemento():
    def __init__(self,elemento, base, altura, moduloElastico, coordenada_xi, coordenada_yi, coordenada_xf, coordenada_yf, conexion):
        
        self.Elem = elemento
        self.b = base
        self.h = altura
        self.E = moduloElastico
        self.cxi = coordenada_xi
        self.cyi = coordenada_yi
        self.cxf = coordenada_xf
        self.cyf = coordenada_yf
        self.conex = conexion
        # Propiedades del elemento
        self.A = self.Area()
        self.I = self.Inercia()
        self.L = self.Longitud()
        self.ang = self.Angulo()
        self.l = self.coseno()
        self.m = self.seno()

    # Propiedades del elemento

    def Area(self):
        return self.b * self.h
    def Inercia(self):
        return (self.b * self.h**3) / 12
    def Longitud(self):
        return sqrt((self.cxf - self.cxi)**2 + (self.cyf - self.cyi)**2)
    def Angulo(self):
        b = (self.cxf - self.cxi)
        h = (self.cyf - self.cyi)
        try:
            angulo = atan(h/b)*180/pi
        except ZeroDivisionError:
            angulo = 90
        return angulo

    def seno(self):
        return sin(radians(self.ang))

    def coseno(self):
        return cos(radians(self.ang))        



In [3]:

E  = 21*(1000**2)

tbl_Elem = [
    
  # Elem, Ni, Nf, b, h, E,
    ["1", 1, 2, 0.4, 0.4, E],
    ["2", 2, 3, 0.3, 0.3, E],
    ["3", 2, 5, 0.2, 0.5, E],
    ["4", 3, 6, 0.2, 0.5, E],
    ["5", 4, 5, 0.4, 0.4, E],
    ["6", 5, 6, 0.3, 0.3, E],
    ["7", 5, 7, 0.2, 0.5, E],
    ["8", 7, 9, 0.2, 0.5, E],
    ["9", 8, 9, 0.4, 0.4, E],


    ]
    
tbl_Nod = [
    #Nodo, cx, cy
    [1, 0, 0],
    [2, 0, 4],
    [3, 0, 7],
    [4, 5.2, 0],
    [5, 5.2, 4],
    [6, 5.2, 7],
    [7, 7.8, 4],
    [8, 10.1, 1.5],
    [9, 10.1, 4],

    ]


tbl_Fuer = [
    #Nod, fx, fy, Mz
    [7, 0, -20, 0],

  
    ]

tbl_Fuer_W = [
    #Elem, Li,Lf, Wi, Wf
    ["1", 0, 2, 0, ((10*4)/7)],
    ["2",2, 7,  ((10*4)/7), 10],
    ["3",0, 5.2,  -8, -8], #"""CASO ANALIZAR######################
    ["4", 0, 5.2, -8, -8],
    ["7",0, 2.6,  -8, -8,],
    ["8",0, 2.3,  -8, -8,],

    ]

tbl_Rest = [
        #Tipo de Apoyo, Nodo
        # Simple
        # Articulado
        # Empotrado
    ["Empotrado", 1],
    ["Empotrado", 4,],
    ["Empotrado", 8,],

    ]


"""====Diccionarios===="""
# ==============Diccionarios de elementos==================#

diccionarioE = {}

for i in range(len(tbl_Elem)):
    Elem = tbl_Elem[i][0]
    Ni = tbl_Elem[i][1]
    Nf = tbl_Elem[i][2]
    b = tbl_Elem[i][3]
    h = tbl_Elem[i][4]
    E = tbl_Elem[i][5]   
    diccionarioE[Elem] = [Ni, Nf, b, h, E]

# ==============Diccionarios de nodos==================#
    
diccionarioN = {}

for i in range(len(tbl_Nod)):
    Nod = tbl_Nod[i][0]
    cx = tbl_Nod[i][1]
    cy = tbl_Nod[i][2]
    diccionarioN[Nod] = [cx, cy]
    

In [10]:
elementos = [] # Almacena todos los elementos con sus propiedades

def ext_elem(): # Elementos fuera del analisis

    for i in range(len(tbl_Elem)):
        Ni = tbl_Elem[i][1]
        Nf = tbl_Elem[i][2]
        b = tbl_Elem[i][3]
        h = tbl_Elem[i][4]
        Ele = tbl_Elem[i][0]    
        conexion = [Ele, Ni, Nf]
        xi = diccionarioN[Ni][0]
        yi = diccionarioN[Ni][1]
        xf = diccionarioN[Nf][0]
        yf = diccionarioN[Nf][1]     
        barra = Elemento(Ele,b, h, E, xi, yi, xf, yf, conexion)
        elementos.append(barra) # Almacena elemento
    return elementos


elementos = ext_elem()
        

In [42]:
sub = 1
coordenadas = []
coor_sub_elementos = []
cheq = []
for i in range(len(tbl_Elem)):
    coor_p = []
    Elemento = elementos[i]
    L = Elemento.L
    L_unitaria = L / sub
    Ni = Elemento.conex[1]
    Nf = Elemento.conex[2]
    cxi = Elemento.cxi
    cyi = Elemento.cyi
    cxf = Elemento.cxf
    cyf = Elemento.cyf
    angulo = Elemento.ang
    cont = 0
    cx = 0
    cy = 0
    for j in range(sub+1):
        if angulo == 0: 
            cy = Elemento.cyi
            cx = Elemento.cxi + L_unitaria*j
            coor_p.append([cx,cy])
            
        if cont < sub:  
            if angulo == 0: 
                cont += 1 
                cy = Elemento.cyi
                cx = Elemento.cxi + L_unitaria*j
                cheq.append([cx,cy])
                #if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                    #coordenadas.append([cx,cy])
                    
        # Elementos verticales
        if angulo == 90:
            cx = Elemento.cxi
            cy = Elemento.cyi + L_unitaria*j            
            cheq.append([cx,cy])
            coor_p.append([cx,cy])
            #if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                #coordenadas.append([cx,cy])
        # Elementos inclinados crecientes
        
        elif angulo > 0 and ang < 90:
            """estos fracmento forman l_uni en un triangulo rectangulo"""
            Ly_uni =  L_unitaria* sqrt(1/(1+(1/(tan(radians(ang))**2)))) # Longitud del fracmento en y
            Lx_uni = Ly_uni/tan(radians(ang)) # Longitud del fracmento en x
            cx = Elemento.cxi + Lx_uni*j
            cy = Elemento.cyi + Ly_uni*j
            
            #print("j=",j, "Ly= ",Ly_uni,"Lx= ",Lx_uni, "cx= ",cx, "cy= ", cy)
            cheq.append([cx,cy])
            coor_p.append([cx,cy])
            #if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                #coordenadas.append([cx,cy])
                
        # Elementos inclinados decrecientes
        elif angulo < 0:
            Ly_uni =  L_unitaria* sqrt(1/(1+(1/(tan(radians(ang))**2)))) # Longitud del fracmento en y
            Lx_uni = Ly/tan(radians(ang)) # Longitud del fracmento en x
            cx = Elemento.cxi - Lx_uni*j
            cy = Elemento.cyi - Ly_uni*j
            
            cheq.append([cx,cy])
            coor_p.append([cx,cy])
            #if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                #coordenadas.append([cx,cy])
        if [cx, cy] not in coordenadas: # Se chequea si la coord ya esta incluida
                    coordenadas.append([cx,cy])
    coor_sub_elementos.append(coor_p)
coordenadas

[[0, 0.0],
 [0, 4.0],
 [0, 7.0],
 [5.2, 4],
 [5.2, 7],
 [5.2, 0.0],
 [7.8, 4],
 [10.1, 4],
 [10.1, 1.5]]

In [43]:
for i in coordenadas:
    print(i, coordenadas.count(i),  "veces")
len(coordenadas)

[0, 0.0] 1 veces
[0, 4.0] 1 veces
[0, 7.0] 1 veces
[5.2, 4] 1 veces
[5.2, 7] 1 veces
[5.2, 0.0] 1 veces
[7.8, 4] 1 veces
[10.1, 4] 1 veces
[10.1, 1.5] 1 veces


9

In [31]:
for i in coordenadas:
    if coordenadas.count(i) > 1:
        #coordenadas.index(i)
        print("indice:", coordenadas.index(i))
len(coordenadas)



indice: 6
indice: 8
indice: 6
indice: 8
indice: 18
indice: 18


22

In [36]:
set(coordenadas)

TypeError: unhashable type: 'list'